In [1]:
!pip install py4j pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 716.9 kB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=54dcce8885d83ff8e39673722b2f4ec247816b2b67be2a22f3f8dc8c1ecd6151
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('clustercode').getOrCreate()
dataset = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/data/seeds_dataset.csv',header=True,inferSchema=True)

In [3]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [4]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [5]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [6]:
assembler = VectorAssembler(inputCols=dataset.columns,outputCol='features')
final_data = assembler.transform(dataset)

In [7]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features',outputCol='scaledFeatures',)

In [11]:
scaler_model = scaler.fit(final_data)
final_data = scaler_model.transform(final_data)

In [13]:
final_data.head()

Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))

In [19]:
kmeans = KMeans(featuresCol='scaledFeatures',k=3)

In [22]:
model = kmeans.fit(dataset=final_data)

In [26]:
print('ClusterCenters')
centers = model.clusterCenters()
print(centers)

ClusterCenters
[array([ 4.91309043, 10.92012526, 37.32658724, 12.37724251,  8.59393872,
        1.8226071 , 10.35389957]), array([ 6.32636687, 12.38115343, 37.39222755, 13.9206997 ,  9.75485787,
        2.41428142, 12.28078861]), array([ 4.0648023 , 10.14242485, 35.82143905, 11.81918014,  7.51855717,
        3.19361875, 10.40520609])]


In [28]:
from numpy.lib.function_base import select
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows

